In [14]:
import numpy as np
from scipy.io import loadmat
import networkx as nx
from scipy.stats import norm
import matplotlib.pyplot as plt
from numpy.linalg import det, inv
import math
import time
import pingouin as pg
import pandas as pd

import numpy as N, networkx as nx
import sys 
from networkx import DiGraph
from itertools import combinations, permutations, chain
from networkx.algorithms.cycles import find_cycle, simple_cycles
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
import itertools 

Import functions from the principal notebook problem1.ipynb


In [15]:
from ipynb.fs.full.problem1 import complete_graph, cond_part_corr_normal, threshold, find_v_struct, is_unshielded_collider
from ipynb.fs.full.problem1 import statistical_orientation, find_colliders, logical_orientation, is_dag, completely_connected_directed_graph
#from ipynb.fs.full.problem1 import find_colliders_also_linked, find_tails, find_cycles, check_add_edge_cycle, avoid_cycles
#from ipynb.fs.full.problem1 import undirected_edges, equivalence_classes, num_equivalence_class
from ipynb.fs.full.problem1meek import part_corr_pingouin

In [ ]:
n = 10000
nodes_many = list(range(n))
print(f"Time to create the complete undirect graph of {n} nodes:")
      
start = time.time()
complete_graph(nodes_many)
end = time.time()
print(f"Implemented function {end - start} sec")

start = time.time()
nx.complete_graph(nodes_many)
end = time.time()
print(f"Networkx function {end - start} sec")

In [16]:
data = loadmat("Data.mat")['D']
cov = np.cov(data.T)

import pandas as pd

xi = 0
xj = 1
S = [2,3,4,5,6,7]

start = time.time()
part1 = cond_part_corr_normal (cov,[xi,xj],S)
end = time.time()
print(f"Partial correlation using the gaussian property {end - start} sec")

start = time.time()
part2 = part_corr_pingouin (data,[xi,xj],S)
end = time.time()
print(f"Partial correlation using Pingouin {end - start} sec")

print("\nWe can see that the results are almost the same")
print( part1 , part2)

Partial correlation using the gaussian property 0.001993417739868164 sec
Partial correlation using Pingouin 0.04825234413146973 sec

We can see that the results are almost the same
0.0009481858635809481 0.000948185863580827


### . Unshielded colliders

This part of the problem requires to add edges, when possible, without creating new colliders. An important feature that this algorithm must have (as well as the other algorithms) is that it must work iteratively: if we add an edge we must recheck everything, because this added edge can help to add others and so on. We must be able to stop when we cannot add any other edges. Let's see how it works with an example

In [ ]:
# directed
dG4 = nx.DiGraph()
dG4.add_edges_from(([3,1],[6,1]))

# undirected
G4 = nx.Graph()
G4.add_edges_from(([6,1], [1,2], [2,1], [3,1], [2,3], [5,4], [1,5]))

pos = nx.planar_layout(G4)
nx.draw(G4, pos=pos, with_labels=True)

In [ ]:
nx.draw(dG4, pos=pos, with_labels=True)

### . Avoid Big cycles

In this case we want to prove that the algorithm is able to direct also edges in undirected cycles with more than 3 edges

In [ ]:
# directed
dG1 = nx.DiGraph()
dG1.add_edges_from(([1,2],[2,3],[3,4],[4,5]))

# undirected
G1 = nx.Graph()
G1.add_edges_from(([1,2],[2,3],[3,4],[4,5], [5,1],[3,5]))

pos = nx.spring_layout(G1)
nx.draw(G1, pos=pos, with_labels=True)

In [ ]:
nx.draw(dG1, pos=pos, with_labels=True)

In [ ]:
avoid_cycles(G1, dG1)

nx.draw(dG1, pos=pos, with_labels=True)

We not that when the algorithm analyse the big cycle [1, 5, 4, 3, 2] is able to ignore inner edges and add add the right edge 1->5

In [ ]:
nx.draw(dG1, pos=pos, with_labels=True)

### . Avoid Big cyles and use new edges in next loops

In [ ]:
# directed
dG11 = nx.DiGraph()
dG11.add_edges_from(([1,2],[2,3],[3,4],[5,1]))

# undirected
G11 = nx.Graph()
G11.add_edges_from(([1,2],[2,3],[3,4],[4,5], [5,1],[3,5]))

pos = nx.planar_layout(G11)
nx.draw(G11, pos=pos, with_labels=True)

In [ ]:
nx.draw(dG11, pos=pos, with_labels=True)

In [ ]:
avoid_cycles(G11, dG11)

nx.draw(dG11, pos=pos, with_labels=True)

In this case the algorithm is able to firstly analyse the big cycle, adding the right edge, and then, considering the new information, orient also the inner edge 3-5

### . Count equivalence classes indipendent case

Count the number of equivalence classes in a case when we have more than one missing edge

In [ ]:
# directed
dG2 = nx.DiGraph()
dG2.add_edges_from(([1,2],[2,3],[4,3],[4,2]))

# undirected
G2 = nx.Graph()
G2.add_edges_from(([1,2],[2,3],[3,4],[4,2], [1,5], [4,6]))

fig, ax = plt.subplots(figsize=(5, 5))
pos = nx.spring_layout(G2)
nx.draw(G2, pos=pos, with_labels=True)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
nx.draw(dG2, pos=pos, with_labels=True)

In this case the two undirected edges (1-5 and 6-4) should be indipendend from each other. So direct one of them should not influence the direction of the other. We should obtain 4 equivalnce classes:

In [ ]:
equi_classes = []
num_equivalence_class(G2, dG2)

ok! They are four. let's see if they are the one we expect:

In [ ]:
equi_classes = []
a = equivalence_classes(equi_classes, G2, dG2)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
nx.draw(a[0], pos=pos, with_labels=True)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
nx.draw(a[1], pos=pos, with_labels=True)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
nx.draw(a[2], pos=pos, with_labels=True)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
nx.draw(a[3], pos=pos, with_labels=True)

okay :)

### . Count equivalence classes dipendent case

Count the number of equivalence classes in harder cases

In [ ]:
# directed
dG3 = nx.DiGraph()
dG3.add_edges_from(([1,2],[2,3],[4,3]))

# undirected
G3 = nx.Graph()
G3.add_edges_from(([1,2],[2,3],[3,4],[4,2], [4,1]))

fig, ax = plt.subplots(figsize=(5, 5))
pos = nx.spring_layout(G3)
nx.draw(G3, pos=pos, with_labels=True)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
nx.draw(dG3, pos=pos, with_labels=True)

In this case the two undirected edges (1-4 and 2-4) should be dependend from each other. So, direct one of them should influence the direction of the other. Edge 2-4 is 'free': both sides are okay in terms of cycles and colliders. However, once we direct it, 1-4 is not free. For this reason we have (thinking about it) that if 2->4 we must have 1->4 to avoid cycles, while if 4->2 also 1-4 is free. Therefore, we should find 3 cases. Let's see if the code works 

In [ ]:
equi_classes = []
num_equivalence_class(G3, dG3)

ok! They are four. let's see if they are the one we expect:

In [ ]:
equi_classes = []
a = equivalence_classes(equi_classes, G3, dG3)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
nx.draw(a[0], pos=pos, with_labels=True)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
nx.draw(a[1], pos=pos, with_labels=True)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
nx.draw(a[2], pos=pos, with_labels=True)

okay :)